In [1]:
import torch
from math import sqrt
from time import time
import fire
from torch.autograd import Variable
from models import VectorNet
from config import DefaultConfig
from data import VectorNetDataset, collate
from torch.utils.data import DataLoader
import warnings

Using backend: pytorch


In [2]:
loss_tranform = lambda cost: sqrt(cost * 1)

In [ ]:
def train(**kwargs):
    '''
    训练
    '''
    opt.parse(kwargs)
    if opt.load_model_path:
        model.load(opt.load_model_path)
    if opt.use_gpu: model.cuda()

    torch.multiprocessing.set_sharing_strategy('file_system')
    train_data = VectorNetDataset(opt.train_data_root, train=True)
    val_data = VectorNetDataset(opt.train_data_root, train=False)
    train_dataloader = DataLoader(train_data, opt.batch_size, shuffle=True, num_workers=opt.num_workers,
                                  collate_fn=collate)
    val_dataloader = DataLoader(val_data, opt.batch_size, shuffle=False, num_workers=opt.num_workers,
                                collate_fn=collate)

    criterion = torch.nn.MSELoss()
    lr = opt.lr
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=opt.weight_decay)
    TrainingLoss = []
    ValLoss = []
    preloss = 1e100
    for epoch in range(opt.max_epoch):
        losses = 0
        num = 0
        start_time = time()
        for ii, (data, label) in enumerate(train_dataloader):
            # input = Variable(data)
            target = Variable(label)
            if opt.use_gpu:
                data['Agent'] = data['Agent'].to(torch.device('cuda:0'))
                for g in data['Map']:
                    g = g.to(torch.device('cuda:0'))
                data['Agentfeature'] = Variable(data['Agentfeature']).cuda()
                for feature in data['Mapfeature']:
                    feature = Variable(feature).cuda()
                target = target.cuda()
            if len(data['Map']) == 0:
                continue
            optimizer.zero_grad()
            score = model(data['Agent'], data['Map'], data['Agentfeature'], data['Mapfeature'], data['Mapmask'])
            loss = criterion(score.double().reshape(-1, 60), target.double())
            loss.backward()
            optimizer.step()
            losses += loss.data
            num += 1
            if num % 10 == 0:
                print(f"epoch: {epoch}, batch num: {num}, current batch loss: {loss_tranform(loss.data):.4f} m, ", \
                      f"time lapses: {time() - start_time:.1f} s")
        # model.save()
        # TrainingLoss.append(losses / num)
        TrainingLoss.append(loss_tranform(losses / num))
        # ValLoss.append(val(model, val_dataloader))
        ValLoss.append(loss_tranform(val(model, val_dataloader)))
        print(f"epoch: {epoch}, val loss: {ValLoss[-1]}, train loss: {TrainingLoss[-1]}")

        if losses / num > preloss:
            lr = lr * opt.lr_decay

        preloss = losses / num
    model.save('new.pth')

In [ ]:
@torch.no_grad()
def val(model, dataloader):
    '''
    计算模型在验证集上的准确率等信息，用以辅助训练
    '''
    model.eval()
    losses = 0
    num = 0
    criterion = torch.nn.MSELoss()
    for ii, (data, label) in enumerate(dataloader):
        # input = Variable(data)
        target = Variable(label)
        if opt.use_gpu:
            data['Agent'] = data['Agent'].to(torch.device('cuda:0'))
            for g in data['Map']:
                g = g.to(torch.device('cuda:0'))
            data['Agentfeature'] = data['Agentfeature'].cuda()
            for feature in data['Mapfeature']:
                feature = Variable(feature).cuda()
            target = target.cuda()
        if len(data['Map']) == 0:
            continue
        score = model(data['Agent'], data['Map'], data['Agentfeature'], data['Mapfeature'], data['Mapmask'])
        loss = criterion(score.double(), target.double())
        losses += loss.data
        num += 1
        if num % 10 == 0:
            # print(num)
            pass
    model.train()
    # print('eval:',losses/num)
    return losses / num

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
a = [1,2,3,4]
b = list()

In [7]:
def test(**kwargs):
    '''
    测试（inference）
    '''
    opt.parse(kwargs)
    if opt.load_model_path:
        model.load(opt.load_model_path)
    else:
        model.load('new.pth')
    torch.multiprocessing.set_sharing_strategy('file_system')
    test_data = VectorNetDataset(opt.test_data_root, test=True)
    test_dataloader = DataLoader(test_data, opt.batch_size, shuffle=False, num_workers=opt.num_workers,
                                 collate_fn=collate)
    criterion = torch.nn.MSELoss()
    model.eval()
    losses = 0
    num = 0
    for ii, (data, label) in enumerate(test_dataloader):
        # input = Variable(data)
        target = Variable(label)
        if opt.use_gpu:
            # input = input.cuda()
            target = target.cuda()
        if len(data['Map']) == 0:
            continue
        score = model(data['Agent'], data['Map'], data['Agentfeature'], data['Mapfeature'], data['Mapmask'])
        loss = criterion(score.double(), target.double())
        
        print(target.flatten() - score.flatten())
        # print(score.shape)
        losses += loss.data
        num += 1
    model.train()
    print('ade:', losses / num)
    return

opt = DefaultConfig()
model = VectorNet(5, 64, 60)
warnings.filterwarnings("ignore")
test()

user config:
env default
model VectorNet
train_data_root ./data/train/forecasting_train_head_1000
test_data_root ./data/test/test_sample
load_model_path None
batch_size 1
use_gpu False
num_workers 0
print_freq 20
result_file result.csv
max_epoch 10
lr 0.1
lr_decay 0.95
weight_decay 0.0001
parse <bound method DefaultConfig.parse of <config.DefaultConfig object at 0x7feda9171940>>
tensor([-0.1448,  0.1183,  0.1809,  0.0362, -0.1068,  0.2759, -0.1188,  0.0544,
         0.0619,  0.1468, -0.1626, -0.0111, -0.1741,  0.2380,  0.0556,  0.1187,
        -0.0760,  0.0560, -0.1073,  0.1088, -0.3876,  0.1599,  0.0188,  0.1167,
         0.2804,  0.1503,  0.0866,  0.0526,  0.1421,  0.1710,  0.0132,  0.2780,
        -0.0826,  0.1083,  0.0523,  0.0628,  0.0046, -0.5321, -0.2627, -0.0369,
        -0.2835,  0.0601,  0.0025,  0.1628,  0.0793,  0.2022,  0.0458,  0.0892,
         0.1145,  0.0805,  0.0129,  0.0837, -0.1587,  0.0274, -0.0246,  0.1749,
        -0.0081,  0.0761, -0.0080,  0.1638], grad_fn=<SubB

In [4]:
# %matplotlib inline
score

NameError: name 'score' is not defined

In [ ]:
def help():
    '''
    打印帮助的信息 
    '''
    print('help')

In [ ]:
# if __name__ == "__main__":
#     opt = DefaultConfig()
#     model = VectorNet(5, 64, 60)
#     # fire.Fire()
#     warnings.filterwarnings("ignore")
#     # train()
#     test()